In [ ]:
import random as r
import numpy as np
import matplotlib.pyplot as plt
import csv
import collections as col
import os
import time

In [ ]:
# FUNCTIONS ---------------------------------------------------------------------------------------

# Reads in data from a given .csv file
def readData(file):
    inFile = open(file,'r')
    reader = csv.reader(inFile,delimiter = ',')

    polished = [line for line in reader if (line != [] and line != ['',''])]
            
    inFile.close()
    
    del(polished[0])
    
    polished.sort()
    
    return polished


# Updates partisan asymmetry objective function
def update_PA(FD):
    FD_copy = FD.copy()
    curve_Dem = []
    all_won = False
    all_lost = False
    
    # Increase average vote-share until dems win all seats, recording breakpoints for seat gains
    while not all_won:
        change_increase = [(0.5 - fd) for fd in FD_copy if fd < 0.5]
        if len(change_increase) == 0:
            all_won = True
            voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
            seatShare_Dem = (len(FD_copy) - len(change_increase))/(len(FD_copy))
            curve_Dem.append([voteShare_Dem,seatShare_Dem])
            continue
            
        delta_increase = min(change_increase)
        
        voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
        seatShare_Dem = (len(FD_copy) - len(change_increase))/(len(FD_copy))
        curve_Dem.append([voteShare_Dem,seatShare_Dem])
        
        FD_copy = [(fd + delta_increase) for fd in FD_copy] # Increase district vote-shares by enough for dems to win one more seat
        
    current = curve_Dem[0]
    del(curve_Dem[0])
    FD_copy = FD.copy()
    
    # Indicator to not record point in first loop iteration, since it's not a breakpoint
    goAhead = False
    
    # Decrease average vote-share until dems lose all seats, recording breakpoints for seat losses
    while not all_lost:
        change_decrease = [(fd - 0.5) for fd in FD_copy if fd > 0.5]
        if len(change_decrease) == 0:
            all_lost = True
            voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
            seatShare_Dem = (len(change_decrease)+1)/(len(FD_copy))
            curve_Dem.append([voteShare_Dem,seatShare_Dem])
            continue
            
        delta_decrease = min(change_decrease)
        
        voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
        seatShare_Dem = (len(change_decrease)+1)/(len(FD_copy))
        if goAhead:
            curve_Dem.append([voteShare_Dem,seatShare_Dem])
            
        goAhead = True
        
        FD_copy = [(fd - delta_decrease) for fd in FD_copy]
    
    
    curve_Dem.sort()
    
    # Create rep curve from dem curve
    curve_Rep = [[1-cd[0],1-cd[1]+(1/len(FD_copy))] for cd in curve_Dem]
    curve_Rep.sort()
    
    # Calculate area between curves
    integ = 0
    for i in range(0,len(curve_Dem)):
        dem = curve_Dem[i]
        rep = curve_Rep[i]
        rect = (1.0/(len(FD_copy)))*(abs(dem[0]-rep[0]))
        integ += rect
        
        
    # Plot dem and rep curves
#     x = [cd[0] for cd in curve_Dem]
#     y = [cd[1]-1.0/(len(FD_copy)) for cd in curve_Dem]
#     x.append(1.0)
#     y.append(1.0)
#     plt.step(x,y,'b',linewidth=.5,label='Party A')
    
#     x = [cd[0] for cd in curve_Rep]
#     y = [cd[1]-1.0/(len(FD_copy)) for cd in curve_Rep]
#     x.append(1.0)
#     y.append(1.0)
#     plt.step(x,y,'r--',linewidth=.5,label='Party B')
#     plt.axis([0,1,0,1])
#     plt.legend()
#     plt.xlabel('Average Vote-Share')
#     plt.ylabel('Seat-Share')
#     plt.show()
    
    return (integ)


# Initializes dictionary of wasted votes in each district (nominal)
def initialize_wasted(ND,NR):
    
    # Store wasted votes for each district
    WASTED_NOMINAL = {}

    for i in range(0,len(ND)):
        if ND[i] >= NR[i]:
            WASTED_NOMINAL[i+1] = .5*(ND[i]-3*NR[i])
        else:
            WASTED_NOMINAL[i+1] = .5*(3*ND[i]-NR[i])
                
    return WASTED_NOMINAL


# Faster update of nominal efficiency gap objective function
def update_EG_fast():

    sumWasted = 0.0
    for i in range(1,numDistricts):
        sumWasted += wastedNominal[i]

    return (round(abs(sumWasted)/totalVotes,10))


# Update of nominal efficiency gap objective function during an iteration
def update_EG_fast_it(From,Too,FDFrom,FDToo,TVFrom,TVToo):
    
    NDFrom = FDFrom*TVFrom
    NRFrom = (1-FDFrom)*TVFrom
    
    NDToo = FDToo*TVToo
    NRToo = (1-FDToo)*TVToo
    
    if NDFrom >= NRFrom:
        wastedFrom = .5*(NDFrom-3*NRFrom)
    else:
        wastedFrom = .5*(3*NDFrom-NRFrom)
        
        
    if NDToo >= NRToo:
        wastedToo = .5*(NDToo-3*NRToo)
    else:
        wastedToo = .5*(3*NDToo-NRToo)
        

    sumWasted = 0.0
    for i in range(1,numDistricts):
        if i == From:
            sumWasted += wastedFrom
        elif i == Too:
            sumWasted += wastedToo
        else:
            sumWasted += wastedNominal[i]
            

    return (round(abs(sumWasted)/totalVotes,10)),wastedFrom,wastedToo


# initializes dictionary of wasted votes in each district for each vote-share shift
def initialize_wastedShift(FD,TV):
    
     # Shifted Efficiency Gap objective function (MO)
    percentShiftsNeg = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0]
    percentShiftsPos = [0.01,0.02,0.03,0.04,0.05]
    #allVotes = sum(TV)
    WASTED_ALL = {}

    for s in percentShiftsNeg:
        fracDem_shift = [max(fd+s,0.0) for fd in FD]
        numDem_shift = []
        numRep_shift = []
        for i in range(0,len(TV)):
            numDem_shift.append(fracDem_shift[i]*TV[i])
            numRep_shift.append((1-fracDem_shift[i])*TV[i])
        
        #wasted_shift = []
        for i in range(0,len(numDem_shift)):
            if numDem_shift[i] >= numRep_shift[i]:
                #wasted_shift.append(.5*(numDem_shift[i]-3*numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(numDem_shift[i]-3*numRep_shift[i])
            else:
                #wasted_shift.append(.5*(3*numDem_shift[i]-numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(3*numDem_shift[i]-numRep_shift[i])
        
 
    for s in percentShiftsPos:
        fracDem_shift = [min(fd+s,1.0) for fd in FD]
        numDem_shift = []
        numRep_shift = []
        for i in range(0,len(TV)):
            numDem_shift.append(fracDem_shift[i]*TV[i])
            numRep_shift.append((1-fracDem_shift[i])*TV[i])
            
        #wasted_shift = []
        for i in range(0,len(numDem_shift)):
            if numDem_shift[i] >= numRep_shift[i]:
                #wasted_shift.append(.5*(numDem_shift[i]-3*numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(numDem_shift[i]-3*numRep_shift[i])
            else:
                #wasted_shift.append(.5*(3*numDem_shift[i]-numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(3*numDem_shift[i]-numRep_shift[i])
                
    return WASTED_ALL


# Faster update of shifted efficiency gap objective function
def update_ShiftedEG_fast():
    
    percentShifts = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]
    EG_shift = []
    
    for s in percentShifts:
        sumWasted = 0.0
        for i in range(1,numDistricts):
            sumWasted += wastedShift[(i,s)]
            
        EG_shift.append(round(abs(sumWasted)/totalVotes,10))
        
#     print('-----')
#     for val in EG_shift:
#         print(val)
        
    return max(EG_shift)
    #return EG_shift[5] # Quick change to get nominal EG value!


# Returns updated efficiency gap value during an iteration
def update_ShiftedEG_fast_it(From,Too,FDFrom,FDToo,TVFrom,TVToo):
    
    percentShiftsNeg = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0]
    percentShiftsPos = [0.01,0.02,0.03,0.04,0.05]
    percentShifts = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]
    EG_shift = []
    
    wastedFrom = {}
    wastedToo = {}
    
    for s in percentShiftsNeg:
        numDemFrom_shift = (max(FDFrom+s,0.0))*TVFrom
        numRepFrom_shift = (1-max(FDFrom+s,0.0))*TVFrom
        
        numDemToo_shift = (max(FDToo+s,0.0))*TVToo
        numRepToo_shift = (1-max(FDToo+s,0.0))*TVToo
        
        if numDemFrom_shift >= numRepFrom_shift:
            wastedFrom[s] = .5*(numDemFrom_shift-3*numRepFrom_shift)
        else:
            wastedFrom[s] = .5*(3*numDemFrom_shift-numRepFrom_shift)
            
        if numDemToo_shift >= numRepToo_shift:
            wastedToo[s] = .5*(numDemToo_shift-3*numRepToo_shift)
        else:
            wastedToo[s] = .5*(3*numDemToo_shift-numRepToo_shift)
            
            
    for s in percentShiftsPos:
        numDemFrom_shift = (min(FDFrom+s,1.0))*TVFrom
        numRepFrom_shift = (1-min(FDFrom+s,1.0))*TVFrom
        
        numDemToo_shift = (min(FDToo+s,1.0))*TVToo
        numRepToo_shift = (1-min(FDToo+s,1.0))*TVToo
        
        if numDemFrom_shift >= numRepFrom_shift:
            wastedFrom[s] = .5*(numDemFrom_shift-3*numRepFrom_shift)
        else:
            wastedFrom[s] = .5*(3*numDemFrom_shift-numRepFrom_shift)
            
        if numDemToo_shift >= numRepToo_shift:
            wastedToo[s] = .5*(numDemToo_shift-3*numRepToo_shift)
        else:
            wastedToo[s] = .5*(3*numDemToo_shift-numRepToo_shift)
            
            
    for s in percentShifts:
        sumWasted = 0.0
        for i in range(1,numDistricts):
            if i == From:
                sumWasted += wastedFrom[s]
            elif i == Too:
                sumWasted += wastedToo[s]
            else:
                sumWasted += wastedShift[(i,s)]
            
        EG_shift.append(round(abs(sumWasted)/totalVotes,10))
        
    return max(EG_shift),wastedFrom,wastedToo
    #return EG_shift[5],wastedFrom,wastedToo # Quick change to get nominal EG value!
    
    
# Checks contiguity of district distFrom upon removal of node n
def checkContig(n,BOTH,BOTH_aug):
    
    # Check contiguity of distFrom upon removal of chosen using a breadth-first search
    isPath = True
    x = BOTH[0] # Recall, nodes is N(n) intersect V(district(n))
    discovered = [x]
    bfsQ = col.deque()
    bfsQ.append(x)
    found = False
    while len(bfsQ) > 0:
        v = bfsQ.popleft()
        vNbhd = set(neighborhoods[v])
        #vBoth = [x for x in vNbhd if data[x] == From] # For basic BFS
        vBoth = list(vNbhd.intersection(BOTH_aug)) # For aug-nbhd BFS (geo-graph)
        if n in vBoth:
            vBoth.remove(n)

        for w in vBoth:
            if w not in discovered:
                discovered.append(w)
                bfsQ.append(w)

    for y in BOTH:
        if y not in discovered:
            isPath = False
            break
            
    return isPath


# Checks if moving node n from district From to district Too creates a hole
def checkHole(n,From,Too):
    
    isPath = True
    disconnect = False

    # Increment if Too and otherDist will have an additional node pair
    # Decrement if From and otherDist lose a node pair
    for vert in neighborhoods_aug[n]:
        otherDist = data[vert]
        if Too != otherDist:
            auxH[Too,otherDist] += 1
            auxH[otherDist,Too] += 1

        if From != otherDist:
            auxH[From,otherDist] -= 1
            auxH[otherDist,From] -= 1
            
            # if edge deleted, signal for possible disconnect
            if (auxH[From,otherDist] == 0) or (auxH[otherDist,From] == 0):
                disconnect = True
                #numDisconnect += 1


    if disconnect:

        # Gather aux neighborhoods
        auxNeighborhoods = [[] for p in range(0,numDistricts)]

        for i in range(0,numDistricts):
            for j in range(0,numDistricts):
                if auxH[i,j] > 0:
                    auxNeighborhoods[i].append(j)
                    
        # Remove Too and all Too adjacencies
        auxNeighborhoods[Too] = []

        for i in range(0,len(auxNeighborhoods)):
            temp = auxNeighborhoods[i]
            if Too in temp:
                temp.remove(Too)
                auxNeighborhoods[i] = temp

        # Check that new auxiliary graph H is connected
        isPath = True
        zones = [i for i in range(0,numDistricts)]
        zones.remove(Too)
        x = zones[0]
        discovered = [x]
        bfsQ = col.deque()
        bfsQ.append(x)
        found = False
        while len(bfsQ) > 0:
            v = bfsQ.popleft()
            auxNbhd = auxNeighborhoods[v]

            for w in auxNbhd:
                if w not in discovered:
                    discovered.append(w)
                    bfsQ.append(w)

        for y in zones:
            if y not in discovered:
                isPath = False
                break


        # Reject chosen node if moving it creates a hole/surrounded zone
        if not isPath:

            # Revert auxH back to previous
            for vert in neigborhoods_aug[n]:
                otherDist = data[vert]
                if Too != otherDist:
                    auxH[Too,otherDist] -= 1 # Decrement here to undo the increment!
                    auxH[otherDist,Too] -= 1

                if From != otherDist:
                    auxH[From,otherDist] += 1 # Increment here to undo the decrement!
                    auxH[otherDist,From] += 1

                    

    return isPath


# Checks that moving node n from district From to district Too satisfies constraints and improves objective
def FlipCheck(From,Too,NODE,passed):
    
    # Use bool to keep track of whether objective/constraints are satisfied
    checks = True
    
    # Record updates in case move is accepted
    updatedValues = {}
    
    # Determine intersection of N(chosen node)/N_aug(chosen node) and V(District(chosen node))
    both = [n for n in neighborhoods[NODE] if data[n] == From]
    both_aug = [n for n in neighborhoods_aug[NODE] if data[n] == From]
    
    # Reject move if node is the only unit in its district
    if len(both) == 0:
        checks = False
        updatedValues['why'] = 'empty'

    # Check pop objective/constraint, if applicable
    if checks:

        # Determine new district populations
        newPopFrom = distPop[From] - pop[NODE]
        newPopToo = distPop[Too] + pop[NODE]

        percentFrom = abs(1-(newPopFrom/meanDistPop))
        percentToo = abs(1-(newPopToo/meanDistPop))

        dev = max(percentFrom,percentToo)
        
        updatedValues['pop'] = [newPopFrom,newPopToo] # Keep track of district populations even when not objective/constraint

        if ((objective == 'pop') or ('pop' in constraints)) and (dev > passed['pop']):
            checks = False
            updatedValues['why'] = 'pop'


    # If previous checks pass, check perim objective/constraint, if applicable
    if checks:
            
        perimFrom = 0
        perimToo = 0
        for nb in neighborhoods[NODE]:
            if data[nb] == From:
                perimFrom += edgesLength[(NODE,nb)]
            elif data[nb] == Too:
                perimToo += edgesLength[(NODE,nb)]

        newPerimFrom = distPerimeter[From] + (2*perimFrom - perimeters[NODE])
        newPerimToo = distPerimeter[Too] + (perimeters[NODE] - 2*perimToo)

        updatedValues['perim'] = [newPerimFrom,newPerimToo] # Keep track of district perimeters even when not objective/constraint
        
        newPerimeter = newPerimFrom + newPerimToo

        if ((objective == 'perim') or ('perim' in constraints)) and (newPerimeter > passed['perim']):
            checks = False
            updatedValues['why'] = 'perim'
                
                
    # If previous checks pass, check demo constraint, if applicable
    if checks:
        if 'demo' in constraints:
            
            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((fracW[From]*distPop[From]) - popW[NODE])/(distPop[From]-pop[NODE])
            newFracWToo = ((fracW[Too]*distPop[Too]) + popW[NODE])/(distPop[Too]+pop[NODE])
            
            newFracBFrom = ((fracB[From]*distPop[From]) - popB[NODE])/(distPop[From]-pop[NODE])
            newFracBToo = ((fracB[Too]*distPop[Too]) + popB[NODE])/(distPop[Too]+pop[NODE])
            
            newFracLFrom = ((fracL[From]*distPop[From]) - popL[NODE])/(distPop[From]-pop[NODE])
            newFracLToo = ((fracL[Too]*distPop[Too]) + popL[NODE])/(distPop[Too]+pop[NODE])

            countB = 0
            countL = 0
            countW = 0

            if newFracWFrom < 0.5:
                if newFracBFrom > max(newFracWFrom,newFracLFrom):
                    countB += 1
                elif newFracLFrom > max(newFracWFrom,newFracBFrom):
                    countL += 1
                else:
                    countW += 1

            if newFracWToo < 0.5:
                if newFracBToo > max(newFracWToo,newFracLToo):
                    countB += 1
                elif newFracLToo > max(newFracWToo,newFracBToo):
                    countL += 1
                else:
                    countW += 1

            if countB != passed['numB'] or countL != passed['numL'] or countW != passed['numW']:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWToo]
                updatedValues['fracB'] = [newFracBFrom,newFracBToo]
                updatedValues['fracL'] = [newFracLFrom,newFracLToo]
                
                
                
    # If previous checks pass, check whole constraint, if applicable, and only if distToo is already wholly contained within 1 county
    if checks:
        if ('whole' in constraints) and (len(passed['whole']) == 1):
            
            if NODE[0:5] not in passed['whole']:
                checks == False
                updatedValues['why'] = 'whole'
                    
                    
    # If previous checks pass, check contiguity -- mandatory!
    if checks:
        checks = checkContig(NODE,both,both_aug)
        if not checks:
            updatedValues['why'] = 'contig'
        
        
    # Update votes
    if checks:
        
        v = votes[NODE]
        newNumDemFrom = numDem[From] - v[0]
        newNumRepFrom = numRep[From] - v[1]
        newNumDemToo = numDem[Too] + v[0]
        newNumRepToo = numRep[Too] + v[1]
        
        newTVFrom = newNumDemFrom + newNumRepFrom
        newTVToo = newNumDemToo + newNumRepToo
        
        newFDFrom = newNumDemFrom/newTVFrom
        newFDToo = newNumDemToo/newTVToo
        
        updatedValues['ND'] = [newNumDemFrom,newNumDemToo]
        updatedValues['NR'] = [newNumRepFrom,newNumRepToo]
        
        
    # If previous checks pass, check cmpttv objective/constraint, if applicable
    if checks:
        if (objective == 'cmpttv') or ('cmpttv' in constraints):

            # Calculate margins of victory in both districts
            if newTVFrom != 0.0 and newTVToo != 0.0:
                newCmpttvFrom = abs(newNumDemFrom-newNumRepFrom)/newTVFrom
                newCmpttvToo = abs(newNumDemToo-newNumRepToo)/newTVToo
            else:
                newCmpttvFrom = 1.5
                newCmpttvToo = 1.5

            if 'cmpttv' in constraints:

                # Check whether same number of competitive districts (<= 10% margin of victory)
                if passed['cmpttv'][0] <= cmpttv_margin and passed['cmpttv'][1] <= cmpttv_margin:
                    if newCmpttvFrom > cmpttv_margin or newCmpttvToo > cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

                elif passed['cmpttv'][0] <= cmpttv_margin or passed['cmpttv'][1] <= cmpttv_margin:
                    if newCmpttvFrom > cmpttv_margin and newCmpttvToo > cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                    elif newCmpttvFrom <= cmpttv_margin and newCmpttvToo <= cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                        
                elif passed['cmpttv'][0] > cmpttv_margin and passed['cmpttv'][1] > cmpttv_margin:
                    if newCmpttvFrom <= cmpttv_margin or newCmpttvToo <= cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

            else:
                
                # Check that number of competitive districts did not decrease (<= 10% margin of victory)
                if passed['cmpttv'][0] <= cmpttv_margin and passed['cmpttv'][1] <= cmpttv_margin:
                    if newCmpttvFrom > cmpttv_margin or newCmpttvToo > cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

                elif passed['cmpttv'][0] <= cmpttv_margin or passed['cmpttv'][1] <= cmpttv_margin:
                    if newCmpttvFrom > cmpttv_margin and newCmpttvToo > cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                    
                    
    # If previous checks pass, check mm objective/constraint, if applicable
    if checks:
        if (objective == 'mm') or ('mm' in constraints):

            if newTVFrom != 0.0 and newTVToo != 0.0:
                newFD = [fd for fd in fracDem]
                newFD[From] = newFDFrom
                newFD[Too] = newFDToo

                mm = abs((np.median(newFD[1:])) - (sum(newFD[1:])/(numDistricts-1)))

            else:
                mm = 1.5

            if mm > passed['mm']:
                checks = False
                updatedValues['why'] = 'mm'
            else:
                updatedValues['mm'] = mm
        
        
    # If previous checks pass, check eg objective/constraint, if applicable
    if checks:
        if (objective == 'eg') or ('eg' in constraints):

            if newTVFrom != 0.0 and newTVToo != 0.0:
                eg,wFrom,wToo = update_EG_fast_it(From,Too,newFDFrom,newFDToo,newTVFrom,newTVToo)
            else:
                eg = 1.5

            if eg > passed['eg']:
                checks = False
                updatedValues['why'] = 'eg'
            else:
                updatedValues['eg'] = [eg,wFrom,wToo]
                
                
    
    # If previous checks pass, check eg_shift objective/constraint, if applicable
    if checks:
        if (objective == 'eg_shift') or ('eg_shift' in constraints):

            if newTVFrom != 0.0 and newTVToo != 0.0:
                seg,wFrom,wToo = update_ShiftedEG_fast_it(From,Too,newFDFrom,newFDToo,newTVFrom,newTVToo)
            else:
                seg = 1.5

            if seg > passed['eg_shift']:
                checks = False
                updatedValues['why'] = 'seg'
            else:
                updatedValues['seg'] = [seg,wFrom,wToo]
                

                
    # If previous checks pass, check pa objective/constraint, if applicable
    if checks:
        if (objective == 'pa') or ('pa' in constraints):

            if newTVFrom != 0.0 and newTVToo != 0.0:
                newFD = [fd for fd in fracDem]
                newFD[From] = newFDFrom
                newFD[Too] = newFDToo
                pa = update_PA(newFD[1:])
            else:
                pa = 1.5


            if pa > passed['pa']:
                checks = False
                updatedValues['why'] = 'pa'
            else:
                updatedValues['pa'] = pa
                
                
    # If previous checks pass, check for hole -- mandatory!
    if checks:
        checks = checkHole(NODE,From,Too)
        if not checks:
            updatedValues['why'] = 'hole'

        
    # Return values
    if checks:
        updatedValues['accept'] = True
    else:
        updatedValues['accept'] = False
        
    return updatedValues
    
    
# READ IN ALL PARAMETERS --------------------------------------------------------------------------------

# Record start time
start = time.time()

# Create boolean variable to report whether user input is valid
VALID = True

# Read in parameters
parameterFile = open('Flip_PARAMETERS.csv','r')
readerParam = csv.reader(parameterFile,delimiter=',')
parameters = [line for line in readerParam]

# Folder of district plan files
BigFolder = parameters[0][1]
print('Folder with maps: ',BigFolder)

if os.path.isdir(BigFolder):
    maps = [p for p in os.listdir(BigFolder+'/') if not p.startswith('.')]
    maps.sort()
else:
    VALID = False
    print('\n-----Folder does not exist-----\n')

if len(parameters[0]) > 2 and os.path.isdir(BigFolder):
    if parameters[0][2] != '':
        maps = []
        for val in parameters[0][2:]:
            if val == '':
                continue
            elif os.path.isfile(BigFolder+'/'+val):
                maps.append(val)
            else:
                VALID = False
                print('\n-----File does not exist-----\n')
                break

        print('Maps:\n',maps)
    

# Folder with state data
StateFolder = parameters[1][1]
print('Folder with state data: ',StateFolder)

if not os.path.isdir(StateFolder):
    VALID = False
    print('\n-----Folder does not exist-----\n')
else:
    files = [t for t in os.listdir(StateFolder+'/') if not t.startswith('.')]
    files.sort()


# Folder for output
OutputFolder = parameters[2][1]
print('Folder for output: ',OutputFolder)

if not os.path.isdir(OutputFolder):
    VALID = False
    print('\n-----Folder does not exist-----\n')


# Assign number of iterations
K = int(parameters[3][1])
print('Number of iterations: ',K)


# Determine objective function
objective = parameters[4][1]

ValidObjectives = ['none','pop','perim','eg','eg_shift','mm','pa','cmpttv']

if objective in ValidObjectives:
    print('Objective: ',objective)
else:
    VALID = False
    print('\n-----',objective,' is not a valid objective-----\n')

    
# Determine constraints
constraints = parameters[5][1:]
constraints = [c for c in constraints if (c != '')]
print('Constraints:\n',constraints)

ValidConstraints = ['pop','perim','eg','eg_shift','mm','pa','cmpttv','demo','whole']

for c in constraints:
    if c not in ValidConstraints:
        VALID = False
        print('\n-----',c,' is not a valid constraint-----\n')


# Assign thresholds
popMinThresh = float(parameters[6][1])
perimAdd = float(parameters[7][1])
EGThresh = float(parameters[8][1])
EGShiftThresh = float(parameters[9][1])
MMThresh = float(parameters[10][1])
PAThresh = float(parameters[11][1])

if 'pop' in constraints:
    print('Population threshold: ',popMinThresh)
    
if 'perim' in constraints:
    print('Value added to current perimeter for perimeter threshold: ',perimAdd)
    
if 'eg' in constraints:
    print('EG threshold: ',EGThresh)
    
if 'eg_shift' in constraints:
    print('EGShift threshold: ',EGShiftThresh)
    
if 'mm' in constraints:
    print('MM threshold: ',MMThresh)
    
if 'pa' in constraints:
    print('PA threshold: ',PAThresh)
    

# Assign edge weights
edgeBonus = float(parameters[12][1])
edgePenalty = float(parameters[13][1])

print('Edge bonus: ',edgeBonus)
print('Edge penalty: ',edgePenalty)


# Determine if user wants objective convergence
converge = parameters[14][1]

if converge == 'yes':
    converge = True
    print('Convergence')
    if len(parameters[14]) > 2:
        if parameters[14][2] != '':
            epsilon = float(parameters[14][2])
        else:
            VALID = False
            print('\n----- Did not provide convergence threshold -----\n')
    else:
        VALID = False
        print('\n----- Did not provide convergence threshold -----\n')
elif converge == 'no':
    converge = False
    print('No convergence')
else:
    VALID = False
    print('\n-----',converge,' is not a valid convergence option-----\n')
    

# Determine if user wants these maps run repeatedly
repeated_runs = int(parameters[15][1])

# Assign cmpttv margin
cmpttv_margin = float(parameters[16][1])

    
# Generate and report random seed
SEED = 0
while (SEED % 2 == 0):
    SEED = r.randrange(1000000001,9999999999)
#SEED = 5132890803
r.seed(SEED)
print('\nSeed = ', SEED)


# If user input is invalid, stop program
if not VALID:
    print('\nPROGRAM STOPPED DUE TO INVALID USER INPUT')
    
# If all user input is valid, run the program
else:
    
# STATE INFO -----------------------------------------------------------------------------------------------

    # Read in state info
    adj = readData(StateFolder+'/'+files[0])       # Unit adjacency
    unitInfo = readData(StateFolder+'/'+files[1])  # Unit info
    if len(files) > 2:
        forbidden = readData(StateFolder+'/'+files[2]) # Forbidden unit adjacency
    else:
        forbidden = []


    #forbidden = []


    # Gather dictionaries of unit info
    ids = {} # GEOIDs
    pop = {} # total population
    votes = {} # dem/rep votes
    popW = {} # white population
    popB = {} # Black/African American population
    popL = {} # Lat/Hisp population
    
    for u in unitInfo:
        ids[u[0]] = 0
        pop[u[0]] = int(u[1])
        votes[u[0]] = [float(u[2]),float(u[3])]
        popW[u[0]] = int(u[4])
        popB[u[0]] = int(u[5])
        popL[u[0]] = int(u[6])

            
    print('Number of units: ',len(ids))

    # Insert dummy node 0
    ids['0'] = 0
    pop['0'] = 0
    votes['0'] = [0.0,0.0]
    popW['0'] = 0
    popB['0'] = 0
    popL['0'] = 0

    print('State Info')
    

# ADJACENCY ------------------------------------------------------------------------------------------------

    # Make cleaned adjacency dictionary
    edgesLength = {}

    # Populate adjacency matrix with length of shared segment for normal adjacency and -1 for aug adjacency

    # Normal adjacency
    for e in adj:
        if e[2] != '0' and e[2] != '0.0' and ([e[0][0:5],e[1][0:5]] not in forbidden) and ([e[1][0:5],e[0][0:5]] not in forbidden) and ([e[0],e[1]] not in forbidden) and ([e[1],e[0]] not in forbidden):
            
            if e[0] != 'outside' and e[1] != 'outside':
                edgesLength[(e[0],e[1])] = float(e[2])/1000 # Meters --> Kilometers
                edgesLength[(e[1],e[0])] = float(e[2])/1000 # Record both so don't have to check each time that the edge pair is in the dictionary
            
            elif e[1] == 'outside' and e[0] != 'outside':
                edgesLength[(e[0],'0')] = float(e[2])/1000 # Record adjacency to outside of state
                edgesLength[('0',e[0])] = float(e[2])/1000

    # Aug adjacency - need to finish normal before doing aug, to skip over aug if two units already have normal            
    for e in adj:
        if e[2] == '0' or e[2] == '0.0' and ([e[0][0:5],e[1][0:5]] not in forbidden) and ([e[1][0:5],e[0][0:5]] not in forbidden) and ([e[0],e[1]] not in forbidden) and ([e[1],e[0]] not in forbidden):
            if (e[0],e[1]) not in edgesLength and (e[1],e[0]) not in edgesLength:
                
                if e[0] != 'outside' and e[1] != 'outside':
                    edgesLength[(e[0],e[1])] = -1
                    edgesLength[(e[1],e[0])] = -1 # Record both so don't have to check each time that the edge pair is in the dictionary
                
                elif e[1] == 'outside' and e[0] != 'outside':
                    edgesLength[(e[0],'0')] = -1 # Record aug adjacency to outside of state
                    edgesLength[('0',e[0])] = -1


    print('Adjacency')
#     print('Number of cut-vertex candidates: ',len(cutCandidates))
    
    
# NEIGHBORHOODS -----------------------------------------------------------------------------------

    # Gather a list of neighbors and aug neighbors for every unit
    neighborhoods = {}
    neighborhoods_aug = {}

    for i in ids:
        neighborhoods[i] = []
        neighborhoods_aug[i] = []

    for pair in edgesLength: # Now that an edge is in edgesLength as both (a,b) and (b,a), don't need double neighborhood statements
        #if pair[0] != '0' and pair[1] != '0':
        if edgesLength[pair] > 0:
            neighborhoods[pair[0]].append(pair[1])
            neighborhoods_aug[pair[0]].append(pair[1])
        elif edgesLength[pair] < 0:
            neighborhoods_aug[pair[0]].append(pair[1])


    print('Neighborhoods')
    
# NODE PERIMETERS ---------------------------------------------------------------------------------

    # Calculate perimeter of each node (does NOT include state perimeter segments)
    perimeters = {}

    for i in ids:
        perimSum = 0
        for nb in neighborhoods[i]:
            if nb != '0':
                perimSum += edgesLength[(i,nb)]

        perimeters[i] = perimSum

    print('Node perimeters')
    
# STATE PERIMETER ---------------------------------------------------------------------------------

    # Calculate state perimeter (remains constant throughout)
    statePerim = 0
    
    for pair in edgesLength:
        if pair[0] == '0': # only need one, since edges are double-recorded
            statePerim += edgesLength[pair]
            
    print('State perimeter')
    
    
# CHECK ALL CUT-VERTEX CANDIDATES -----------------------------------------------------------------

    # Check contiguity of aug. neighborhood of candidate upon removal of candidate using a breadth-first search

    cutVertices = {}

    #for chosen in cutCandidates:
    for chosen in ids:
        if chosen != '0':
            chosen_nbhd = neighborhoods[chosen].copy()
            if '0' in chosen_nbhd:
                chosen_nbhd.remove('0')
            chosen_nbhd_aug = neighborhoods_aug[chosen].copy()
            if '0' in chosen_nbhd_aug:
                chosen_nbhd_aug.remove('0')
            chosen_nbhd_aug = set(chosen_nbhd_aug)

            isPath = True
            x = chosen_nbhd[0]
            discovered = [x]
            bfsQ = col.deque()
            bfsQ.append(x)
            found = False
            while len(bfsQ) > 0:
                v = bfsQ.popleft()
                vNbhd = neighborhoods[v].copy()
                if '0' in vNbhd:
                    vNbhd.remove('0')
                vNbhd = set(vNbhd)
                vBoth = list(vNbhd.intersection(chosen_nbhd_aug)) # For aug-nbhd BFS
                if chosen in vBoth:
                    vBoth.remove(chosen)

                for w in vBoth:
                    if w not in discovered:
                        discovered.append(w)
                        bfsQ.append(w)


            for y in chosen_nbhd:
                if y not in discovered:
                    isPath = False
                    break


            # Add chosen to dictionary of cut-vertices if aug. neighborhood will become discontiguous
            if not isPath:
                cutVertices[chosen] = [] # this list will contain the nodes it surrounds


    print('Number of cut-vertices: ',len(cutVertices))
    

# IDENTIFY UNITS SURROUNDED BY CUT-VERTICES -------------------------------------------------------

    # Start breadth-first search at a neighbor of a cut-vertex.
    # If search continues to reach more than 75 nodes, starting node is not surrounded.
    # If search stops before reaching 75 nodes, starting node is surrounded.

    for chosen in cutVertices:
        
#         print('chosen: ',chosen)
#         print(neighborhoods[chosen])
        
        chosen_nbhd = neighborhoods[chosen].copy()
        if '0' in chosen_nbhd:
            chosen_nbhd.remove('0')
#         print(chosen_nbhd)
            
        for x in chosen_nbhd:
            
#             print('x: ',x)

            isSurrounded = True
            discovered = [x]
            bfsQ = col.deque()
            bfsQ.append(x)
            found = False
            while len(bfsQ) > 0:
                v = bfsQ.popleft()
                vBoth = neighborhoods[v].copy()
                if '0' in vBoth:
                    vBoth.remove('0')
                if chosen in vBoth:
                    vBoth.remove(chosen)

                for w in vBoth:
                    if w not in discovered:
                        discovered.append(w)
                        bfsQ.append(w)
#                         print('w: ',w)


                if len(discovered) > 75: # Large number due to odd peninsula of blocks in MO
                    isSurrounded = False
                    break
                    
#             print('isSurrounded: ',isSurrounded)

            # Record all nodes that cut-vertex "chosen" surrounds
            if isSurrounded:
                for d in discovered:
                    if d not in cutVertices[chosen]:
                        cutVertices[chosen].append(d)


    # Keep list of all surrounded units
    surrounded = []

    for cv in cutVertices:
        for unit in cutVertices[cv]:
            if unit not in surrounded:
                surrounded.append(unit)


    # If a unit is both a cut-vertex and surrounded, remove it from cutVertices
    for unit in surrounded:
        if unit in cutVertices:
            del cutVertices[unit]

    print('Number of cut-vertices (post-cleaning): ',len(cutVertices))
    
    
#     surrounded_outFile = open('IL_SurroundedTracts.csv','w')
#     surrounded_writer = csv.writer(surrounded_outFile,delimiter=',')
    
#     surrounded_writer.writerow(['GEOID20','Cut/Surrounded'])
    
#     for cv in cutVertices:
#         surrounded_writer.writerow([cv,'0'])
#         for surr in cutVertices[cv]:
#             surrounded_writer.writerow([surr,'1'])
        
#     surrounded_outFile.close()


# MERGE DATA OF SURROUNDED UNITS WITH SURROUNDING UNIT --------------------------------------------

    for chosen in cutVertices:
        for unit in cutVertices[chosen]:

            pop[chosen] += pop[unit]
            votes[chosen][0] += votes[unit][0]
            votes[chosen][1] += votes[unit][1]
            popW[chosen] += popW[unit]
            popB[chosen] += popB[unit]
            popL[chosen] += popL[unit]

            ids[unit] = 1
            del pop[unit]
            del votes[unit]
            del popW[unit]
            del popB[unit]
            del popL[unit]

            for nb in neighborhoods[unit]:
                neighborhoods[nb].remove(unit)

            for nb in neighborhoods_aug[unit]:
                neighborhoods_aug[nb].remove(unit)

            del neighborhoods[unit]
            del neighborhoods_aug[unit]


    print('Units merged')
    
    
# ITERATE THROUGH EACH DISTRICT PLAN --------------------------------------------------------------

    # Open output file to link initial map with optimized map

#     outMap = open(OutputFolder+'/Flip_'+StateFolder+'_'+objective+'_'+str(SEED)+'Seed_MapList.csv','w')
#     writerMap = csv.writer(outMap,delimiter=',')
#     writerMap.writerow(['Initial Map',objective+' Final Map'])

    outRejects = open(OutputFolder+'/Flip_'+StateFolder+'_'+objective+'Obj_'+str(SEED)+'Seed_Rejections.csv','w')
    writerRejects = csv.writer(outRejects,delimiter=',')
    reasons = ['empty','pop','demo','perim','contig','hole',
               'whole','eg','eg_shift','mm','pa','cmpttv']
    writerRejects.writerow(['Final Plan']+reasons)
    

    for z in range(0,repeated_runs):

        for plan in maps:

            # Record start time for a particular map
            startMap = time.time()

            print('\nGiven district map: ',plan)

# DISTRICTS ---------------------------------------------------------------------------------------

            # Read in plan data, clean data
            data_raw = readData(BigFolder+'/'+plan)
            data = {}
            numDistricts = 0
            for d in data_raw:
                if ids[d[0]] == 0: # don't record merged units
                    data[d[0]] = int(d[1])
                    if int(d[1]) > numDistricts:
                        numDistricts = int(d[1])

            # Insert dummy node 0 in dummy district 0 (to represent outside the district)
            data['0'] = 0
            numDistricts += 1

            # Count number of counties spanned by each district
            if 'whole' in constraints:

                distCounties = [[] for p in range(0,numDistricts)]
                for i in data:
                    if i[0:5] not in distCounties[data[i]]:
                        distCounties[data[i]].append(i[0:5])
                    

            print('Districts')

    # BORDER NODES ------------------------------------------------------------------------------------

            # Gather units on the border of each district
            borderNodes = [] # List of all border nodes
            borderNodesByDist = {} # Records border nodes for each district (used to calculate initial district perimeters & auxH)
            borderNodesYesNo = {} # For every node, record whether it's a border node (can be adapted to record # of districts node is adjacent to, besides its own)
            
            for i in data:
                borderNodesYesNo[i] = 0

            for i in range(0,numDistricts):
                borderNodesByDist[i] = []

            for node in data:
                dist = data[node]
                for nb in neighborhoods_aug[node]:
                    otherDist = data[nb]
                    if otherDist != dist:
                        borderNodesByDist[dist].append(node)
                        if otherDist != 0 and dist != 0 and edgesLength[(node,nb)] > 0: # Don't want nodes to move to dummy district 0:
                            borderNodes.append(node)
                            borderNodesYesNo[node] = 1
                        break # Exit neighborhood loop if node is already identified as on the border (DON'T need to count units multiple times)
            

            print('# Border nodes: ',len(borderNodes))
            
    # AUXILIARY GRAPH H(G) ----------------------------------------------------------------------------

            # Make auxiliary district graph for hole check
            auxH = {}
            #goodlabels = []
        
            for i in range(0,numDistricts):
                for j in range(0,numDistricts):
                    auxH[(i,j)] = 0
#                     if (i,j) not in goodlabels:
#                         goodlabels.append((i,j))
#                     if (j,i) not in goodlabels:
#                         goodlabels.append((j,i))
                    
            for i in range(0,numDistricts):
                for b in borderNodesByDist[i]:
                    for n in neighborhoods_aug[b]:
                    #for n in neighborhoods[b]:
                        if data[n] != i:
                            auxH[(i,data[n])] += 1
                            

#             for gl in goodlabels:
#                 print(gl,':',auxH[gl])
            
            print('Auxiliary graph')

    # POPULATION --------------------------------------------------------------------------------------

            # Calculate population of each district
            distPop = [0 for i in range(0,numDistricts)]

            for i in data:
                distPop[data[i]] += pop[i]

            meanDistPop = sum(distPop)/(numDistricts-1) #numDistricts-1 bc don't want to include dummy district
            distPop[0] = meanDistPop # Change dummy districts population to the expected

            # Calculate each district's absolute percent difference from expected population
            distPopPercent = [abs(1-(dp)/(meanDistPop)) for dp in distPop]

            popInitial = max(distPopPercent)
            print('Initial max population deviation: ',popInitial)

            valuesPop = [popInitial]
            popDev = popInitial

    # COMPACTNESS -------------------------------------------------------------------------------------

            # Calculate district perimeters
            distPerimeter = [0.0 for i in range(0,numDistricts)]
            distPerimeter[0] = 0.0 # Dummy district perimeter

            for i in range(1,numDistricts):
                for b in borderNodesByDist[i]:
                    for n in neighborhoods[b]:
                        if data[n] != data[b] and edgesLength[(n,b)] > 0 and n != '0':
                            distPerimeter[i] += edgesLength[(n,b)]


            perimInitial = sum(distPerimeter)
            perimInitial += statePerim # Add state perimeter to total perimeter
            print('Initial total perimeter: ',perimInitial)
            valuesPerim = [perimInitial]
            totalPerim = perimInitial

            if objective == 'perim':
                objValue = totalPerim

    # VOTES -------------------------------------------------------------------------------------------

            # Calculate fraction of dem, and number of dem/rep in every district
            fracDem = [0.0 for i in range(0,numDistricts)]
            numDem = [0.0 for i in range(0,numDistricts)]
            numRep = [0.0 for i in range(0,numDistricts)]

            for d in data:
                numDem[data[d]] += votes[d][0]
                numRep[data[d]] += votes[d][1]

            for i in range(1,numDistricts):
                fracDem[i] = numDem[i]/(numDem[i]+numRep[i])

            totalVotesByDistrict = [numDem[i]+numRep[i] for i in range(0,len(numDem))]
            totalVotes = sum(totalVotesByDistrict)


    # FAIRNESS METRICS --------------------------------------------------------------------------------

            # Calculate initial Efficiency Gap value
            if (objective == 'eg') or ('eg' in constraints):

                wastedNominal = initialize_wasted(numDem[1:],numRep[1:])
                EGInitial = update_EG_fast()
                print('Initial EG: ',EGInitial)
                valuesEGNominal = [EGInitial]
                EGNominal = EGInitial

                if objective == 'eg':
                    objValue = EGNominal


            # Calculate initial Shifted Efficiency Gap value
            if (objective == 'eg_shift') or ('eg_shift' in constraints):

                wastedShift = initialize_wastedShift(fracDem[1:],totalVotesByDistrict[1:])
                EGShiftInitial = update_ShiftedEG_fast()
                print('Initial Max EG Shift: ',EGShiftInitial)
                valuesEGShift = [EGShiftInitial]
                EGShift = EGShiftInitial

                if objective == 'eg_shift':
                    objValue = EGShift

                # Slower calculation of EGShift
            #     EGShiftInitial = update_ShiftedEG(fracDem[1:],totalVotesByDistrict[1:])
            #     print('Initial Max EG Shift OLD: ',EGShiftInitial)
            #     valuesEGShift = [EGShiftInitial]
            #     EGShift = EGShiftInitial


            # Calculate initial Median-Mean value
            if (objective == 'mm') or ('mm' in constraints):

                MMInitial = abs((np.median(fracDem[1:])) - (sum(fracDem[1:])/(numDistricts-1))) # abs(Median(FD) - Mean(FD))
                print('Initial MM: ',MMInitial)
                valuesMM = [MMInitial]
                MM = MMInitial

                if objective == 'mm':
                    objValue = MM


            # Calculate initial Partisan Asymmetry value
            if (objective == 'pa') or ('pa' in constraints):

                PAInitial = update_PA(fracDem[1:])
                print('Initial PA: ',PAInitial)
                valuesPA = [PAInitial]
                PA = PAInitial

                if objective == 'pa':
                    objValue = PA


            # Calculate initial competitiveness
            if (objective == 'cmpttv') or ('cmpttv' in constraints):

                compFrac = [abs(2*fd - 1) for fd in fracDem[1:]]
                compFrac.insert(0,0.0)
                CmpttvInitial = sum(compFrac)/(len(compFrac)-1)
                print('Initial avg margin (Cmpttv): ',CmpttvInitial)
                print('Initial max margin: ',max(compFrac))

                countCmpttv = 0
                for cp in compFrac[1:]:
                    if cp <= cmpttv_margin:
                        countCmpttv += 1

                print('Initial number within '+str(100*cmpttv_margin)+'% margin: ',countCmpttv)
                
                numCmpttv = [countCmpttv]
                Cmpttv = countCmpttv

                if objective == 'cmpttv':
                    objValue = Cmpttv


    # MAJORITY-MINORITY -------------------------------------------------------------------------------

            # Calculate the fraction of district pop that is white/Black/Lat
            if 'demo' in constraints:

                fracW = [0.0 for i in range(0,numDistricts)]
                fracB = [0.0 for i in range(0,numDistricts)]
                fracL = [0.0 for i in range(0,numDistricts)]

                for d in data:
                    fracW[data[d]] += popW[d]
                    fracB[data[d]] += popB[d]
                    fracL[data[d]] += popL[d]

                for i in range(0,numDistricts):
                    fracW[i] = float(fracW[i])/distPop[i]
                    fracB[i] = float(fracB[i])/distPop[i]
                    fracL[i] = float(fracL[i])/distPop[i]

                numPlurB = 0
                numPlurL = 0
                numPlurW = 0

                for i in range(1,numDistricts):
                    if fracW[i] < 0.5:
                        if fracB[i] > fracW[i] and fracB[i] > fracL[i]:
                            numPlurB += 1
                        elif fracL[i] > fracW[i] and fracL[i] > fracB[i]:
                            numPlurL += 1
                        else:
                            numPlurW += 1

                print('Initial # plurality-Black MM districts: ',numPlurB)
                print('Initial # plurality-Lat/Hisp MM districts: ',numPlurL)
                print('Initial # plurality-white MM districts: ',numPlurW)


    # ITERATIONS ------------------------------------------------------------------------------------

            finished = False
            k = 0
            numIt = 0
            
            countPop = 0
            
            countRejects = {}
            for re in reasons:
                countRejects[re] = 0

            while not finished:

                # Give user idea of algorithm progress
                k += 1
                #print(k)

                # Check convergence, or complete a pre-determined number of iterations
                if converge and k > 1:
                    if (abs(objValue_Old - objValue) < epsilon):
                        numIt += 1
                        #print('\nobjValue_Old: ',objValue_Old)
                        #print('objValue: ',objValue)
                    else:
                        numIt = 0

                    if numIt >= 10000:
                        finished = True
                        continue

                elif (not converge) and (k >= K):
                    finished = True


                if converge:
                    objValue_Old = objValue


                # Determine unit in consideration for move and its district
                index = int(len(borderNodes)*r.random())
                node = borderNodes[index]
                distFrom = data[node]

                # Choose district that chosen unit can move too
                candidates = []

                for nb in neighborhoods[node]:
                    otherDist = data[nb]
                    if nb != '0' and otherDist != distFrom and otherDist not in candidates:
                        candidates.append(otherDist)

                index = int(len(candidates)*r.random())
                distToo = candidates[index]

#                 print('\nNode: ',node)
#                 print('distFrom: ',distFrom)
#                 print('distToo: ',distToo)


                # Create dictionary of values to pass to checks function
                valuesToPass = {}

                if objective == 'pop':
                    valuesToPass['pop'] = max(distPopPercent[distFrom],distPopPercent[distToo])
                elif 'pop' in constraints:
                    valuesToPass['pop'] = popMinThresh

                if objective == 'perim':
                    valuesToPass['perim'] = distPerimeter[distFrom]+distPerimeter[distToo]
                elif 'perim' in constraints:
                    valuesToPass['perim'] = perimInitial + perimAdd - (totalPerim - distPerimeter[distFrom] - distPerimeter[distToo])

                if objective == 'eg':
                    valuesToPass['eg'] = EGNominal
                elif 'eg' in constraints:
                    valuesToPass['eg'] = EGThresh

                if objective == 'eg_shift':
                    valuesToPass['eg_shift'] = EGShift
                elif 'eg_shift' in constraints:
                    valuesToPass['eg_shift'] = EGShiftThresh

                if objective == 'mm':
                    valuesToPass['mm'] = MM
                elif 'mm' in constraints:
                    valuesToPass['mm'] = MMThresh

                if objective == 'pa':
                    valuesToPass['pa'] = PA
                elif 'pa' in constraints:
                    valuesToPass['pa'] = PAThresh

                if objective == 'cmpttv':
                    valuesToPass['cmpttv'] = [compFrac[distFrom],compFrac[distToo]]
                elif 'cmpttv' in constraints:
                    valuesToPass['cmpttv'] = [compFrac[distFrom],compFrac[distToo]]

                if 'whole' in constraints:
                    valuesToPass['whole'] = distCounties[distToo]

                if 'demo' in constraints:
                    numW = 0
                    numB = 0
                    numL = 0

                    if fracW[distFrom] < 0.5:
                        if fracB[distFrom] > max(fracW[distFrom],fracL[distFrom]):
                            numB += 1
                        elif fracL[distFrom] > max(fracW[distFrom],fracB[distFrom]):
                            numL += 1
                        else:
                            numW += 1

                    if fracW[distToo] < 0.5:
                        if fracB[distToo] > max(fracW[distToo],fracL[distToo]):
                            numB += 1
                        elif fracL[distToo] > max(fracW[distToo],fracB[distToo]):
                            numL += 1
                        else:
                            numW += 1

                    valuesToPass['numW'] = numW
                    valuesToPass['numB'] = numB
                    valuesToPass['numL'] = numL


                # Call flip step check
                newValues = FlipCheck(distFrom,distToo,node,valuesToPass)


                # Once move is accepted/rejected, update everything
                if newValues['accept']:

                    #print('Accepted!\n')

                    # Update full record
                    data[node] = distToo

                    # Update district populations
                    distPop[distFrom] = newValues['pop'][0]
                    distPop[distToo] = newValues['pop'][1]

                    # Update district percent difference from expected population
                    distPopPercent[distFrom] = abs(1-(distPop[distFrom])/(meanDistPop))
                    distPopPercent[distToo] = abs(1-(distPop[distToo])/(meanDistPop))

                    # Record pop objective value
                    popDev = max(distPopPercent)
                    valuesPop.append(popDev)

                    # Update district perimeters
                    distPerimeter[distFrom] = newValues['perim'][0]
                    distPerimeter[distToo] = newValues['perim'][1]
                    totalPerim = sum(distPerimeter) + statePerim
                    valuesPerim.append(totalPerim)

                    if objective == 'perim':
                        objValue = totalPerim

                    # Update borderNodes
                    toAppend = []
                    for vert in neighborhoods[node]:
                        isBorderNode = 0
                        distVert = data[vert]
                        for nb in neighborhoods[vert]:
                            otherDist = data[nb]
                            if otherDist != distVert and otherDist != 0 and distVert != 0:
                                if borderNodesYesNo[vert] == 0:
                                    toAppend.append(vert)
                                isBorderNode = 1
                                break

                        borderNodesYesNo[vert] = isBorderNode

                    
                    newBN = [bn for bn in borderNodes if borderNodesYesNo[bn] == 1]
                    for bn in toAppend:
                        newBN.append(bn)
                        
                    borderNodes = [bn for bn in newBN]
                    newBN = []
                    
                    
                    
#                     # Slower borderNodes update
#                     for vert in neighborhoods[node]:
#                         borderNodesYesNo[vert] = 0
#                         distVert = data[vert]
#                         for nb in neighborhoods[vert]:
#                             otherDist = data[nb]
#                             if otherDist != distVert and otherDist != 0 and distVert != 0:
#                                 borderNodesYesNo[vert] = 1
#                                 break

#                     borderNodes = []
#                     for vert in data:
#                         if borderNodesYesNo[vert] == 1:
#                             borderNodes.append(vert)
                        
                        

                    # Update votes
                    numDem[distFrom] = newValues['ND'][0]
                    numDem[distToo] = newValues['ND'][1]

                    numRep[distFrom] = newValues['NR'][0]
                    numRep[distToo] = newValues['NR'][1]

                    totalVotesByDistrict[distFrom] = numDem[distFrom]+numRep[distFrom]
                    totalVotesByDistrict[distToo] = numDem[distToo]+numRep[distToo]

                    fracDem[distFrom] = numDem[distFrom]/totalVotesByDistrict[distFrom]
                    fracDem[distToo] = numDem[distToo]/totalVotesByDistrict[distToo]

                    # Update % white, Black, Lat/Hisp
                    if 'demo' in constraints:
                        fracW[distFrom] = newValues['fracW'][0]
                        fracB[distFrom] = newValues['fracB'][0]
                        fracL[distFrom] = newValues['fracL'][0]

                        fracW[distToo] = newValues['fracW'][1]
                        fracB[distToo] = newValues['fracB'][1]
                        fracL[distToo] = newValues['fracL'][1]

                    # Update Cmpttv
                    if (objective == 'cmpttv') or ('cmpttv' in constraints):

                        compFrac[distFrom] = abs(2*fracDem[distFrom] - 1)
                        compFrac[distToo] = abs(2*fracDem[distToo] - 1)
                        Cmpttv = len([f for f in compFrac[1:] if f <= cmpttv_margin])
                        numCmpttv.append(Cmpttv)

                        if objective == 'cmpttv':
                            objValue = Cmpttv

                    # Update MM
                    if (objective == 'mm') or ('mm' in constraints):

                        MM = newValues['mm']
                        valuesMM.append(MM)

                        if objective == 'mm':
                            objValue = MM

                    # Update EG
                    if (objective == 'eg') or ('eg' in constraints):

                        EGNominal = newValues['eg'][0]
                        wastedNominal[distFrom] = newValues['eg'][1]
                        wastedNominal[distToo] = newValues['eg'][2]

                        valuesEGNominal.append(EGNominal)  

                        if objective == 'eg':
                            objValue = EGNominal

                    # Updated Shifted EG
                    if (objective == 'eg_shift') or ('eg_shift' in constraints):

                        EGShift = newValues['seg'][0]
                        for s in [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]:
                            wastedShift[(distFrom,s)] = newValues['seg'][1][s]
                            wastedShift[(distToo,s)] = newValues['seg'][2][s]

                        valuesEGShift.append(EGShift)

                        if objective == 'eg_shift':
                            objValue = EGShift

                    # Update PA
                    if (objective == 'pa') or ('pa' in constraints):

                        PA = newValues['pa']
                        valuesPA.append(PA)

                        if objective == 'pa':
                            objValue = PA


                else:
                    #print('Rejected! because ',newValues['why'])
#                     if newValues['why'] == 'hole':
#                         print('hole')
                    
                    countRejects[newValues['why']] += 1
                        
                    if objective == 'pop':
                        valuesPop.append(popDev)
                    elif objective == 'perim':
                        valuesPerim.append(totalPerim)
                    elif objective == 'eg':
                        valuesEGNominal.append(EGNominal)
                    elif objective == 'eg_shift':
                        valuesEGShift.append(EGShift)
                    elif objective == 'mm':
                        valuesMM.append(MM)
                    elif objective == 'pa':
                        valuesPA.append(PA)
                    elif objective == 'cmpttv':
                        numCmpttv.append(Cmpttv)



    # OUTPUT ------------------------------------------------------------------------------------------


            print('Final max population deviation: ',popDev)
            print('Final total perimeter: ',totalPerim)

            if (objective == 'eg') or ('eg' in constraints):
                print('Final EG: ', EGNominal)

            if (objective == 'eg_shift') or ('eg_shift' in constraints):
                print('Final Max EG Shift: ',EGShift)

            if (objective == 'mm') or ('mm' in constraints):
                print('Final MM: ',MM)

            if (objective == 'pa') or ('pa' in constraints):
                print('Final PA: ',PA)

            if (objective == 'cmpttv') or ('cmpttv' in constraints):

                print('Final avg margin (Cmpttv): ',Cmpttv)
                print('Final max margin (Cmpttv): ',max(compFrac))
                countCmpttv = 0
                for cp in compFrac[1:]:
                    if cp <= cmpttv_margin:
                        countCmpttv += 1
                        #print(cp)

                print('Margins <= '+str(100*cmpttv_margin)+'%: ',countCmpttv)

            if 'demo' in constraints:

                numPlurB = 0
                numPlurL = 0
                numPlurW = 0

                for i in range(1,numDistricts):
                    if fracW[i] < 0.5:
                        if fracB[i] > fracW[i] and fracB[i] > fracL[i]:
                            numPlurB += 1
                        elif fracL[i] > fracW[i] and fracL[i] > fracB[i]:
                            numPlurL += 1
                        else:
                            numPlurW += 1

                print('Final # plurality-Black MM districts: ',numPlurB)
                print('Final # plurality-Lat/Hisp MM districts: ',numPlurL)
                print('Final # plurality-white MM districts: ',numPlurW)



            # Record map runtime
#             mapTime = round((time.time()-startMap)/60,2)
#             print('Runtime for this map: ',mapTime,' minutes')
            mapTime = time.time()-startMap
            print('Runtime for this map: ',mapTime,' seconds')


            # Used for output file name
            objectiveString = ''

            if objective == 'none':

                objectiveString = objective + 'Obj'

            elif objective == 'pop':

                objectiveString = objective + 'Obj' + str(round(popDev,4))

                plt.plot(valuesPop)
                plt.xlabel('Iterations')
                plt.ylabel('Max pop dev')
                plt.title('Change in max pop dev over time')
                plt.show()

            elif objective == 'perim':

                objectiveString = objective + 'Obj' + str(int(totalPerim))

                plt.plot(valuesPerim)
                plt.xlabel('Iterations')
                plt.ylabel('Total perimeter')
                plt.title('Change in total perimeter over time')
                plt.show()

            elif objective == 'eg':

                objectiveString = objective + 'Obj' + str(round(EGNominal,5))

                plt.plot(valuesEGNominal)
                plt.xlabel('Iterations')
                plt.ylabel('EG')
                plt.title('Change in EG over time')
                plt.show()

            elif objective == 'eg_shift':

                objectiveString = objective + 'Obj' + str(round(EGShift,5))

                plt.plot(valuesEGShift)
                plt.xlabel('Iterations')
                plt.ylabel('Max EG Shift')
                plt.title('Change in Max EG Shift over time')
                plt.show()

            elif objective == 'mm':

                objectiveString = objective + 'Obj' + str(round(MM,5))

                plt.plot(valuesMM)
                plt.xlabel('Iterations')
                plt.ylabel('MM')
                plt.title('Change in MM over time')
                plt.show()

            elif objective == 'pa':

                objectiveString = objective + 'Obj' + str(round(PA,5))

                plt.plot(valuesPA)
                plt.xlabel('Iterations')
                plt.ylabel('PA')
                plt.title('Change in PA over time')
                plt.show()

            elif objective == 'cmpttv':

                objectiveString = objective + 'Obj' + str(countCmpttv)

                plt.plot(numCmpttv)
                plt.xlabel('Iterations')
                plt.ylabel('Number of Cmpttv Districts')
                plt.title('Change in Number of Cmpttv Districts over time')
                plt.show()



            constraintString = ''

            if 'eg' in constraints:
                constraintString += '_EG'+str(round(EGNominal,5))

            if 'eg_shift' in constraints:
                constraintString += '_EGShift'+str(round(EGShift,5))

            if 'mm' in constraints:
                constraintString += '_MM'+str(round(MM,5))

            if 'pa' in constraints:
                constraintString += '_PA'+str(round(PA,5))

            if 'cmpttv' in constraints:
                constraintString += '_Cmpttv'+str(countCmpttv)

            if 'demo' in constraints:
                constraintString += '_demo'

            if 'whole' in constraints:
                constraintString += '_whole'

            if objective != 'perim':
                constraintString += '_Perim'+str(int(totalPerim))


            # Open and write to output file
            #outFile_str = 'Flip_' + StateFolder + '_Seed'+ str(SEED) +'_It' + str(k)+'_'+str(mapTime)+'minutes'+'_B'+str(edgeBonus)+'_P'+str(edgePenalty)+'_'+objectiveString+constraintString+'.csv'
            outFile_str = 'Flip_' + StateFolder + '_Seed'+ str(SEED) +'_It' + str(k)+'_'+str(mapTime)+'seconds'+'_B'+str(edgeBonus)+'_P'+str(edgePenalty)+'_'+objectiveString+constraintString+'.csv'
            outFile = open(OutputFolder + '/' + outFile_str,'w')

            writer = csv.writer(outFile,delimiter=',')
            writer.writerow(['GEOID','district'])
            for d in data:
                if d != '0':
                    writer.writerow([d,data[d]])
                    if d in cutVertices:
                        for u in cutVertices[d]:
                            writer.writerow([u,data[d]])

            outFile.close()
            
            #writerMap.writerow([plan,outFile_str])
            
            writerRejects.writerow([outFile_str]+[countRejects[val] for val in countRejects])
        
        
        
    #outMap.close()
    outRejects.close()

    print('\nRuntime = ',time.time()-start,' seconds')

In [ ]:
outFile.close()
outRejects.close()